In [1]:
import numpy as np
import tensorflow as tf

sess = tf.InteractiveSession()

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### 1.导入数据

In [2]:
#用tf导入数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/MNIST_data", one_hot=True)
print(mnist.test.labels.shape)
print(mnist.train.labels.shape)

Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz
(10000, 10)
(55000, 10)


### 2.构建网络

In [3]:
def conv2d(x, filter_shape, strides_x, strides_y, padding, name):
    assert padding in ['SAME', 'VALID']
    strides = [1, strides_x, strides_y, 1]
    with tf.variable_scope(name):
        W_conv = tf.get_variable('W', shape=filter_shape)
        b_conv = tf.get_variable('b', shape=[filter_shape[-1]])
        h_conv = tf.nn.conv2d(x, W_conv, strides=strides, padding=padding)
        h_conv_relu = tf.nn.relu(h_conv + b_conv)
    return h_conv_relu

def max_pooling(x, k_height, k_width, strides_x, strides_y, padding='SAME'):
    ksize = [1, k_height, k_width, 1]
    strides = [1, strides_x, strides_y, 1]
    h_pool = tf.nn.max_pool(x, ksize=ksize, strides=strides, padding=padding)
    return h_pool

def dropout(x, keep_prob, name = None):
    return tf.nn.dropout(x, keep_prob=keep_prob, name=name)

def fc(x, in_size, out_size, name, activation=None):
    if activation is not None:
        assert activation in ['relu', 'sigmoid', 'tanh'], 'Wrong activation function'
    with tf.variable_scope(name):
        w = tf.get_variable('w', shape= [in_size, out_size], dtype=tf.float32)
        b = tf.get_variable('b', [out_size], dtype=tf.float32)
        h_fc = tf.nn.xw_plus_b(x, w, b)
        if activation == 'relu':
            return tf.nn.relu(h_fc)
        if activation == 'tanh':
            return tf.nn.tanh(h_fc)
        if activation == 'sigmoid':
            return tf.nn.sigmoid(h_fc)
        else:
            return h_fc


In [4]:
with tf.name_scope('inputs'):
    X_ = tf.placeholder(tf.float32, [None,  784])
    y_ = tf.placeholder(tf.float32, [None, 10])

#把x转化为卷积所需要的格式
X = tf.reshape(X_, [-1, 28, 28, 1])
h_conv1 = conv2d(X, [5, 5, 1, 32], 1, 1, 'SAME', 'conv1')
h_pool1 = max_pooling(h_conv1, 2, 2, 2, 2)

h_conv2 = conv2d(h_pool1, [5, 5, 32, 64], 1, 1, 'SAME', 'conv2')
h_pool2 = max_pooling(h_conv2, 2, 2, 2, 2)

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = fc(h_pool2_flat, 7*7*64, 1024, 'fc1', 'relu')

# dropout: 输出的维度和h_fc1一样，只是随机部分值被值为零
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
h_fc2 = fc(h_fc1_drop, 1024, 10, 'fc2')
y_conv = tf.nn.softmax(h_fc2)

print('X_input:', X_)
print('X:      ', X)
print('y_input:', y_)
print('h_conv1:', h_conv1)
print('h_pool1:', h_pool1)
print('h_conv2:', h_conv2)
print('h_pool2:', h_pool2)
print('h_fc1:  ', h_fc1)

X_input: Tensor("inputs/Placeholder:0", shape=(?, 784), dtype=float32)
X:       Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
y_input: Tensor("inputs/Placeholder_1:0", shape=(?, 10), dtype=float32)
h_conv1: Tensor("conv1/Relu:0", shape=(?, 28, 28, 32), dtype=float32)
h_pool1: Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
h_conv2: Tensor("conv2/Relu:0", shape=(?, 14, 14, 64), dtype=float32)
h_pool2: Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
h_fc1:   Tensor("fc1/Relu:0", shape=(?, 1024), dtype=float32)


### 3.训练和预估

In [6]:
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())
for i in range(10000):
    batch = mnist.train.next_batch(50)
    if i%1000 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            X_:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={X_: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g" %accuracy.eval(feed_dict={
    X_: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.08
step 1000, training accuracy 0.96
step 2000, training accuracy 0.94
step 3000, training accuracy 1
step 4000, training accuracy 1
step 5000, training accuracy 1
step 6000, training accuracy 1
step 7000, training accuracy 0.96
step 8000, training accuracy 1
step 9000, training accuracy 1
test accuracy 0.991
